<a href="https://colab.research.google.com/github/Tsuke007/dseg1_w2/blob/main/Insurance_data_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download dataset from: https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction

In [1]:
!gdown --id 1E0eztrotQ9CPH5glLgONIwxV-M4WqXJ_

Downloading...
From: https://drive.google.com/uc?id=1E0eztrotQ9CPH5glLgONIwxV-M4WqXJ_
To: /content/insurance_cross_sell.zip
6.78MB [00:00, 25.7MB/s]


In [2]:
!unzip insurance_cross_sell.zip

Archive:  insurance_cross_sell.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv                
  inflating: train.csv               


#Pipeline
1. Collect Dataset
2. Clean Data
3. Extract Features
4. Split Training and test
5. Train
6. Evaluate
7. Create Pipeline

#1. Collect Dataset

In [3]:
import pandas as pd
import numpy as np

In [4]:
insured_df = pd.read_csv('train.csv')

In [5]:
insured_df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [6]:
def collect_data():
  insured_df = pd.read_csv('train.csv')
  return insured_df

#2. Clean Data

In [7]:
insured_df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [8]:
insured_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [9]:
insured_df['Response']

0         1
1         0
2         1
3         0
4         0
         ..
381104    0
381105    0
381106    0
381107    0
381108    0
Name: Response, Length: 381109, dtype: int64

In [10]:
insured_df.isna().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

#3. Extract Features

In [11]:
insured_df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [12]:
def extract_feature(df, is_training=True):
  insured_new_df = df.copy()
  label_df = []

  if 'Response' in insured_new_df.columns:
    label_df = insured_new_df['Response']

     # drop unused columns
  if is_training:
    insured_new_df =insured_new_df.drop(['Response','id'],axis=1)
  col_names = ['Gender','Vehicle_Age','Vehicle_Damage']

    # One-Hot Encoding
  dummies_df = pd.get_dummies(insured_new_df[col_names])

   # Merge One-Hot Encoding
  insured_new_df = pd.concat([insured_new_df, dummies_df], axis=1)

  # Drop unused columns (One-Hot Encoding)
  insured_new_df = insured_new_df.drop(col_names, axis=1)


  for col in insured_new_df.columns:
    insured_new_df[col] = pd.to_numeric(insured_new_df[col],errors='coerce')

  return insured_new_df,label_df

In [13]:
insured_new_df,label_df = extract_feature(insured_df)

In [14]:
insured_new_df

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Gender_Female,Gender_Male,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
0,44,1,28.0,0,40454.0,26.0,217,0,1,0,0,1,0,1
1,76,1,3.0,0,33536.0,26.0,183,0,1,1,0,0,1,0
2,47,1,28.0,0,38294.0,26.0,27,0,1,0,0,1,0,1
3,21,1,11.0,1,28619.0,152.0,203,0,1,0,1,0,1,0
4,29,1,41.0,1,27496.0,152.0,39,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381104,74,1,26.0,1,30170.0,26.0,88,0,1,1,0,0,1,0
381105,30,1,37.0,1,40016.0,152.0,131,0,1,0,1,0,1,0
381106,21,1,30.0,1,35118.0,160.0,161,0,1,0,1,0,1,0
381107,68,1,14.0,0,44617.0,124.0,74,1,0,0,0,1,0,1


#4. Train & Test split 

In [15]:
len(insured_new_df)

381109

In [16]:
from sklearn.model_selection import train_test_split
train_insured, test_insured, train_label, test_label = \
          train_test_split(insured_new_df, label_df, test_size=0.2, random_state=42)

In [17]:
len(train_insured)

304887

In [18]:
len(test_insured)

76222

In [19]:
def split_train_test(insured_new_df, label):
  train_insured, test_insured, train_label, test_label = \
          train_test_split(insured_new_df, label, test_size=0.2, random_state=42)
  return train_insured, test_insured, train_label, test_label

In [20]:
train_insured, test_insured, train_label, test_label = \
        split_train_test(insured_new_df, label_df)

#5. Train (Classification)

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [22]:
def train_model(insured_new_df, label):
  model = GradientBoostingClassifier(random_state=2020)
  #model = SVC()
  model.fit(insured_new_df, label)
  return model

In [23]:
model = train_model(train_insured, train_label)

In [24]:
model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2020, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

#6. Evaluation (Classification)

In [25]:
def eval_acc(prediction, actual):
  acc = sum(prediction == actual) / len(actual)
  return acc

In [26]:
pred = model.predict(test_insured)
eval_acc(pred, test_label)

0.8750754375377188

In [27]:
train_insured

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Gender_Female,Gender_Male,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
332803,39,1,15.0,0,52906.0,55.0,227,1,0,1,0,0,0,1
116248,38,1,11.0,0,23038.0,26.0,29,0,1,1,0,0,0,1
255005,22,1,30.0,1,45318.0,152.0,166,0,1,0,1,0,1,0
317474,23,1,41.0,1,29132.0,151.0,277,1,0,0,1,0,1,0
344212,56,1,48.0,0,2630.0,154.0,155,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,24,1,36.0,1,22575.0,152.0,287,1,0,0,1,0,1,0
365838,56,1,35.0,0,41287.0,124.0,298,0,1,1,0,0,0,1
131932,22,1,2.0,0,18857.0,152.0,76,1,0,0,1,0,0,1
146867,44,1,32.0,1,2630.0,156.0,51,0,1,1,0,0,0,1


In [28]:
pred = model.predict(test_insured)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [29]:
test_label

200222    0
49766     0
172201    0
160713    0
53272     0
         ..
258403    0
234155    0
24476     0
60423     0
185839    0
Name: Response, Length: 76222, dtype: int64

#7. Create pipeline (Classification)

In [62]:
def run_pipeline():
  # collect data, clean data
  insured_df = collect_data()

  # Extract Feature
  train_df, test_df = extract_feature(insured_df) 
  
  # split training and test
  train_insured, test_insured, train_label, test_label = \
        split_train_test(insured_new_df, label_df)

  # Evaluation
  pred = model.predict(test_insured)
  acc = eval_acc(pred, test_label)

  return acc

In [63]:
print(run_pipeline())

0.8750754375377188


# Save Model

In [74]:
import pickle
pickle.dump(model, open("Insurance.mod", "wb"))

# Test Model after save

In [75]:
modelcom = pickle.load(open("Insurance.mod", "rb"))
a = modelcom.predict(test_insured)
eval_acc(a, test_label)

0.8750754375377188